# Tourism Sentiment Analysis - TripAdvisor NYC Data Extraction

**Project:** Tourism Sentiment Analysis

**Task:** Data Extraction & Processing

**Dataset Source:** TripAdvisor (SciDB)

**Focus:** NYC, 2022-2025, Hotels

**Source URL:** https://www.scidb.cn/en/file?fid=df2d477ee4830d106a58c14053a57b07

## 1. Setup & Configuration
*Import libraries, set up project paths, create directory structure*

### 1A. Imports & Script Setup
*Load required packages and configure script imports*

In [ ]:
# Core data processing
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

# File handling & web requests
import requests
from pathlib import Path
from openpyxl import load_workbook

# System utilities
import sys

# Bootstrap: Add shared scripts to Python path
def setup_scripts_path():
    """Add shared scripts to Python path"""
    current = Path.cwd()
    for _ in range(10):
        if (current / '.projectroot').exists():
            scripts_dir = current / 'notebooks' / 'shared' / 'scripts'
            if scripts_dir.exists():
                sys.path.insert(0, str(scripts_dir))
                return scripts_dir
        if current.parent == current:
            break
        current = current.parent
    raise FileNotFoundError(
        "Scripts directory not found.\n"
        "Ensure .projectroot exists at project root."
    )

# Setup path and import utilities
scripts_dir = setup_scripts_path()

from project_utils import find_project_root
from data_io import setup_extraction_directories, check_existing_file, check_existing_chunks
from data_validation import print_final_summary, print_storage_summary

print(f"✓ Packages and scripts loaded successfully")

### 1B. Project Root Detection
*Cross-platform function to locate project directory automatically*

**Purpose:** Finds project root by searching for `.projectroot` marker file

**Handles:** Working directory issues, different operating systems, various notebook locations

**Confirmation:** Displays detected path for verification

**Manual Override:** Uncomment line below if auto-detection fails

In [ ]:
# Automatically detect project root
project_root = find_project_root()

In [ ]:
# If auto-detection fails, uncomment and edit the following line:
# project_root = Path('/.../.../.../[tourism_data_project]')

### 1C. Set Project Paths
*Establish standardized directory structure for bronze and silver processing*

**Bronze Structure:** Raw download → chunked conversion → primary filter

**Silver Structure:** Final staging area for gold layer integration

**Auto-creation:** All directories created automatically for new collaborators

In [ ]:
# Create all required directories automatically
dirs = setup_extraction_directories(project_root, 'tripadvisor')

# Access directories throughout notebook
original_dir = dirs['bronze_original']
conversion_dir = dirs['bronze_conversion']
primary_filter_dir = dirs['bronze_primary_filter']
silver_staging = dirs['silver_staging']

print("Directories ready:")
print(f"  Bronze original: {original_dir}")
print(f"  Bronze conversion: {conversion_dir}")
print(f"  Bronze primary filter: {primary_filter_dir}")
print(f"  Silver staging: {silver_staging}")

## 2. Data Acquisition
*Download raw Excel file from ScienceDB using discovered direct API*

<details>
<summary><strong>Manual Download Instructions </strong> (click to expand)</summary>

***If automated download fails:***
1. Visit: https://www.scidb.cn/en/file?fid=df2d477ee4830d106a58c14053a57b07
2. Download file manually
3. Rename to: `tripadvisor_nyc_2022_2025_original.xlsx`
4. Place in: `data/bronze/tripadvisor/00_original_download/`

</details>

In [ ]:
# Set up download path
file_id = "df2d477ee4830d106a58c14053a57b07"
url = f"https://china.scidb.cn/download?fileId={file_id}"
file_name = "tripadvisor_nyc_2022_2025_original.xlsx"
file_path = original_dir / file_name

# Check if file already exists
exists, info = check_existing_file(file_path, file_type='xlsx', show_info=True)

if not exists:
    print(f"Downloading from: {url}...")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(file_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    file_size = file_path.stat().st_size / (1024 * 1024)
    print(f"Download complete: {file_path}")
    print(f"File size: {file_size:.1f} MB")

## 3. Bronze Layer: Raw Data Processing
*Convert Excel to chunked parquet files preserving original structure*

**Input:** `data/bronze/00_original_download/tripadvisor_nyc_2022_2025_original.xlsx` (156.9 MB)

**Output:** `data/bronze/01_raw_conversion/tripadvisor_nyc_raw_chunk_*.parquet` (chunked files)

**Processing:** 5,000-row chunks for memory efficiency

**Purpose:** Preserve complete dataset structure while converting to analysis-friendly format

In [ ]:
# Check if conversion already completed
exists, count = check_existing_chunks(
    conversion_dir,
    pattern="tripadvisor_nyc_raw_chunk_*.parquet"
)

if not exists:
    print("Loading Excel file...")
    wb = load_workbook(file_path, read_only=True)
    ws = wb.active
    header = [str(cell.value) if cell.value is not None else f"col_{i}"
              for i, cell in enumerate(next(ws.iter_rows(min_row=1, max_row=1)))]
    print(f"Columns found: {len(header)}")

    # Convert to parquet chunks
    chunk_size = 5000
    rows = []
    part = 0

    print("Converting to parquet chunks...")
    for row in ws.iter_rows(min_row=2, values_only=True):
        row = list(row[:len(header)])  # truncate any extra columns
        while len(row) < len(header):  # fill missing columns with None
            row.append(None)
        rows.append(row)

        if len(rows) >= chunk_size:
            df = pd.DataFrame(rows, columns=header)
            chunk_filename = f"tripadvisor_nyc_raw_chunk_{part:05d}.parquet"
            pq.write_table(pa.Table.from_pandas(df), conversion_dir / chunk_filename, compression="snappy")
            rows = []
            part += 1

            # Progress indicator every 10 files
            if part % 10 == 0:
                print(f"Processed {part} chunks...")

    # Write remaining rows
    if rows:
        df = pd.DataFrame(rows, columns=header)
        chunk_filename = f"tripadvisor_nyc_raw_chunk_{part:05d}.parquet"
        pq.write_table(pa.Table.from_pandas(df), conversion_dir / chunk_filename, compression="snappy")

    print(f"Conversion complete. Total chunks: {part + 1}")
    print(f"Output location: {conversion_dir}")

## 4. Data Verification & Column Inspection
*Load converted data to verify structure and examine columns before filtering*

**Purpose:** Confirm parquet conversion preserved data integrity

**Check:** Column names, data types, row counts

**Next:** Identify date column format for primary filtering

In [ ]:
# Load a sample chunk to verify conversion
sample_file = conversion_dir / "tripadvisor_nyc_raw_chunk_00000.parquet"
df_sample = pd.read_parquet(sample_file)

# Validation checks - flag conversion issues against expected TripAdvisor structure
expected_chunk_size = 5000
expected_columns = [
    'col_0', 'Unnamed: 0', 'hotel_name', 'id_review', 'title',
    'date', 'location', 'user_name', 'user_link', 'date_of_stay',
    'rating', 'review', 'rating_review', 'n_review_user', 'n_votes_review'
]

print(f"Conversion Validation:")
print("=" * 40)

# 1. Exact shape validation
shape_ok = df_sample.shape == (expected_chunk_size, len(expected_columns))
print(f"{'Shape validated' if shape_ok else 'Shape invalid'}")
print(f"Shape: {df_sample.shape} [Expected ({expected_chunk_size}, {len(expected_columns)})]")

# 2. Exact column validation
columns_ok = list(df_sample.columns) == expected_columns
print(f"{'Columns validated' if columns_ok else 'Columns invalid'}")
print(f"Column structure: {columns_ok}")
if not columns_ok:
    missing = set(expected_columns) - set(df_sample.columns)
    extra = set(df_sample.columns) - set(expected_columns)
    if missing: print(f"   Missing: {missing}")
    if extra: print(f"   Extra: {extra}")

# 3. Sample data display
print(f"\nSample Data Preview:")
print(df_sample[['hotel_name', 'date', 'rating', 'location']].head(3))

# 4. Overall conversion status
all_checks_ok = shape_ok and columns_ok
print(f"\n{'Conversion successful' if all_checks_ok else 'Conversion issues detected'}")

if not all_checks_ok:
    print("Review validation failures above before proceeding")

## 5. Primary Filter: Date Range Selection  
*Filter reviews to 2022-2025 timeframe and consolidate chunks*

**Input:** 84 raw chunks (~500K+ total rows)

**Filter Criteria:** Date contains "2022", "2023", "2024", or "2025"

**Output:** `data/bronze/02_primary_filter/tripadvisor_nyc_2022_2025_date_filtered.parquet`  

**Expected Reduction:** ~90% of data (based on original analysis)

In [ ]:
# Set up primary filter output file
output_file = primary_filter_dir / "tripadvisor_nyc_2022_2025_date_filtered.parquet"

# Check if primary filtering already completed
exists, info = check_existing_file(output_file, file_type='parquet')

if not exists:
    # Load all chunks and apply date filter
    years_keywords = ["2022", "2023", "2024", "2025"]
    chunk_files = sorted(conversion_dir.glob("tripadvisor_nyc_raw_chunk_*.parquet"))

    print(f"Processing {len(chunk_files)} chunks for date filtering...")
    all_filtered_rows = []
    original_total = 0

    for i, chunk_file in enumerate(chunk_files):
        df = pd.read_parquet(chunk_file)
        original_total += len(df)
        date_mask = df["date"].fillna("").apply(lambda x: any(year in str(x) for year in years_keywords))
        filtered_df = df[date_mask]
        all_filtered_rows.append(filtered_df)

        # Progress indicator every 20 files
        if (i + 1) % 20 == 0:
            print(f"Processed {i + 1}/{len(chunk_files)} chunks...")

    # Consolidate filtered data
    print("Consolidating filtered chunks...")
    filtered_df = pd.concat(all_filtered_rows, ignore_index=True)

    kept_count = len(filtered_df)
    removed_count = original_total - kept_count

    # Save consolidated result
    filtered_df.to_parquet(output_file, compression="snappy")

    print(f"\nDate filtering complete:")
    print(f"  Original rows: {original_total:,}")
    print(f"  Rows kept (2022-2025): {kept_count:,}")
    print(f"  Rows removed: {removed_count:,}")
    print(f"  Retention rate: {kept_count/original_total*100:.1f}%")
    print(f"\nSaved to: {output_file}")

    # File size check
    file_size_mb = output_file.stat().st_size / (1024*1024)
    print(f"File size: {file_size_mb:.1f} MB")

## 6. Data Verification & Geographic Filtering
*Load converted data to verify structure and extract NYC hotels using positive filtering*

**Input:** `data/bronze/02_primary_filter/tripadvisor_nyc_2022_2025_date_filtered.parquet` (48,992 rows)

**Strategy:** 
1. Verify date filtering results
2. Positive NYC filtering (hotel names with NYC indicators)
3. Manual cleanup of misclassified hotels

**Expected Output:** ~12,500 rows, ~125 hotels

**Final Location:** `data/silver/tripadvisor/staging/tripadvisor_nyc_2022_2025_final.parquet`

### 6A. Data Verification

*Load and verify primary filtered data before geographic analysis*

In [ ]:
# Load primary filtered data for geographic analysis
primary_filter_file = primary_filter_dir / "tripadvisor_nyc_2022_2025_date_filtered.parquet"

if not primary_filter_file.exists():
    print("[Error] Primary filter file not found - run previous steps first")
    print(f"Expected location: {primary_filter_file}")
else:
    exploration_df = pd.read_parquet(primary_filter_file)

    print("Primary Filter Verification:")
    print("=" * 40)
    print(f"File loaded successfully")
    print(f"Rows: {len(exploration_df):,}")
    print(f"Hotels: {exploration_df['hotel_name'].nunique()}")

    # Verify date filtering worked
    date_samples = exploration_df['date'].dropna().head(10).tolist()
    target_years = ["2022", "2023", "2024", "2025"]
    dates_valid = any(any(year in str(date) for year in target_years) for date in date_samples)
    print(f"\n{'Year filter validated' if dates_valid else 'Year filter not valid'}")
    print(f"Sample dates contain target years: {date_samples[:5]}")

    print(f"\nTop 5 hotels by review count:")
    top_hotels = exploration_df['hotel_name'].value_counts().head(5)
    for hotel, count in top_hotels.items():
        print(f"  • {hotel}: {count:,} reviews")

    print(f"\nReady for geographic filtering")

### 6B. Exploratory Analysis Section (Optional)

**Purpose:** Show analysis process used to develop filtering strategy

**Contains:** Implementation strategies for geographic filtering challenges

**Note:** *Optional section - can skip to Section 7 for direct save*

#### 6B.1 Initial Hotel Name Analysis
*Examine hotel name patterns after date filtering*

In [ ]:
# Load date-filtered data for geographic analysis
primary_filter_dir = bronze_base / "02_primary_filter"
exploration_df = pd.read_parquet(primary_filter_dir / "tripadvisor_nyc_2022_2025_date_filtered.parquet")

print(f"Starting geographic analysis with: {len(exploration_df):,} rows")
print(f"Unique hotels: {exploration_df['hotel_name'].nunique()}")

# Initial hotel name examination
print(f"\n Top 15 hotels by review count:")
top_hotels = exploration_df['hotel_name'].value_counts().head(15)
for hotel, count in top_hotels.items():
    print(f"  • {hotel} ({count:,} reviews)")

# Look for obvious non-NYC patterns
print(f"\n Sample hotel names (checking for international patterns):")
sample_hotels = exploration_df['hotel_name'].value_counts().head(25).index
for hotel in sample_hotels:
    print(f"  • {hotel}")

#### 6B.2 UK Reviewer Concentration Strategy

**Approach:** Use reviewer location patterns to identify misclassified hotels*

**Challenge:** Hotel names alone insufficient (e.g., "SoHo" exists in both NYC and London)

**Adjustment:** Analyze reviewer geographic patterns ([user_...] 'location') to detect misclassified hotels

**Logic:** London hotels will have high concentrations of UK-based reviewers

**Threshold:** Hotels with >60% UK reviewers (min. 10 location entries) flagged for removal

In [ ]:
# Analyze reviewer location patterns to identify non-NYC hotels
print("Analyzing reviewer geographic patterns...")

hotel_stats = []
for hotel_name, group in exploration_df.groupby('hotel_name'):
    location_data = group['location'].fillna('')

    total_reviews = len(group)
    total_with_location = group['location'].notna().sum()
    uk_reviews = location_data.str.contains('United Kingdom|UK|England|Scotland|Wales', case=False).sum()
    shanghai_reviews = location_data.str.contains('Shanghai|China', case=False).sum()

    hotel_stats.append({
        'hotel_name': hotel_name,
        'total_reviews': total_reviews,
        'total_with_location': total_with_location,
        'uk_reviews': uk_reviews,
        'shanghai_reviews': shanghai_reviews
    })

# Convert to analysis DataFrame
hotel_analysis = pd.DataFrame(hotel_stats)
hotel_analysis['uk_percentage'] = (hotel_analysis['uk_reviews'] / hotel_analysis['total_with_location']).fillna(0)
hotel_analysis['shanghai_percentage'] = (hotel_analysis['shanghai_reviews'] / hotel_analysis['total_with_location']).fillna(0)

# Identify problematic hotels
uk_threshold = 0.6
uk_hotels = hotel_analysis[
    (hotel_analysis['uk_percentage'] > uk_threshold) &
    (hotel_analysis['total_with_location'] >= 10)
]

print(f"\nHotels with >{uk_threshold*100:.0f}% UK reviewers: {len(uk_hotels)}")
if len(uk_hotels) > 0:
    print("\n UK-heavy hotels (likely London):")
    uk_display = uk_hotels.nlargest(10, 'uk_percentage')[['hotel_name', 'total_with_location', 'uk_percentage']]
    for _, row in uk_display.iterrows():
        print(f"  • {row['hotel_name']} - {row['uk_percentage']:.1%} UK reviewers ({row['total_with_location']} total)")

#### 6B.3 Positive NYC Filtering Strategy

**Approach:** Identify genuine NYC hotels using location indicators*

**Strategy Shift:** Instead of removing non-NYC, actively identify NYC hotels

**Indicators:** Hotel names containing NYC-specific terms

**Advantage:** Reduces false positives from ambiguous neighborhood names (SoHo, Chelsea, etc.)

**Final Cleanup:** Manual removal of remaining misclassified hotels

In [ ]:
# Apply positive NYC filtering - identify genuine NYC hotels
nyc_indicators = [
    'New York', 'NYC', 'Manhattan', 'Brooklyn', 'Queens', 'Bronx',
    'Times Square', 'Time Square', 'Central Park', 'Wall Street',
    'Midtown', 'Downtown', 'Financial District', 'SoHo', 'NoMad',
    'TriBeCa', 'Upper East', 'Upper West', 'Lower East', 'Herald Square',
    'Penn Station', 'Grand Central', 'JFK', 'LaGuardia', 'Empire State'
]

nyc_pattern = '|'.join(nyc_indicators)
nyc_hotels = exploration_df[exploration_df['hotel_name'].str.contains(nyc_pattern, case=False, na=False)]

print(f"NYC hotels identified: {len(nyc_hotels):,} rows")
print(f"Unique NYC hotels: {nyc_hotels['hotel_name'].nunique()}")

# Check for remaining ambiguous terms that might be misclassified
print(f"\n Top 10 NYC hotels:")
nyc_top = nyc_hotels['hotel_name'].value_counts().head(10)
for hotel, count in nyc_top.items():
    print(f"  • {hotel} ({count:,} reviews)")

# Check for potentially ambiguous hotels needing manual review
ambiguous_terms = ['SoHo', 'Chelsea', 'Greenwich', 'Victoria']
print(f"\n NYC hotels with ambiguous neighborhood terms")
for term in ambiguous_terms:
    matching = nyc_hotels[nyc_hotels['hotel_name'].str.contains(term, case=False, na=False)]
    if len(matching) > 0:
        unique_hotels = matching['hotel_name'].unique()
        print(f"\n  {term}: {len(unique_hotels)} hotels")
        for hotel in unique_hotels[:3]:
            print(f"    • {hotel}")

#### 6B.4 Manual Cleanup of Misclassified Hotels
*Remove remaining London hotels caught by ambiguous neighborhood names*

**Issue:** "SoHo" exists in both NYC and London

**Solution:** Remove clearly London-branded hotels

**Targets:** Hotels with "London" in name or known London hotel chains

In [ ]:
# Manual removal of identified London hotels
london_hotels_to_remove = [
    'The Soho Hotel',                       # London SoHo hotel
    'The Z Hotel Soho',                     # London hotel chain
    'hub by Premier Inn London Soho hotel'  # Explicitly London-branded
]

print(f"Removing London hotels:")
for hotel in london_hotels_to_remove:
    count = nyc_hotels[nyc_hotels['hotel_name'] == hotel].shape[0]
    print(f"  • {hotel} ({count:,} reviews)")

# Apply manual cleanup
final_nyc_df = nyc_hotels[~nyc_hotels['hotel_name'].isin(london_hotels_to_remove)].copy()

print(f"\nManual cleanup complete...")
print(f"Final NYC dataset: {len(final_nyc_df):,} rows")
print(f"Unique hotels: {final_nyc_df['hotel_name'].nunique()}")

# Verify remaining SoHo hotels are legitimate NYC hotels
remaining_soho = final_nyc_df[final_nyc_df['hotel_name'].str.contains('soho', case=False)]['hotel_name'].unique()
print(f"\nRemaining SoHo hotels (verified NYC):")
for hotel in remaining_soho:
    print(f"  • {hotel}")

## 7. Silver Layer: Final Geographic Filter & Save
*Clean, validated approach - works whether exploration was run or not*

**Implementation:** Apply proven NYC filter strategy

**Output:** `data/silver/tripadvisor/staging/tripadvisor_nyc_2022_2025_final.parquet`

In [ ]:
# Set up silver staging output file
output_file = silver_staging / "tripadvisor_nyc_2022_2025_final.parquet"

# Check if final filtering already completed
exists, info = check_existing_file(output_file, file_type='parquet')

if not exists:
    # Load primary filtered data (works whether exploration was run or skipped)
    df_for_filtering = pd.read_parquet(primary_filter_dir / "tripadvisor_nyc_2022_2025_date_filtered.parquet")

    # Apply validated NYC filter strategy
    nyc_indicators = [
        'New York', 'NYC', 'Manhattan', 'Brooklyn', 'Queens', 'Bronx',
        'Times Square', 'Time Square', 'Central Park', 'Wall Street',
        'Midtown', 'Downtown', 'Financial District', 'SoHo', 'NoMad',
        'TriBeCa', 'Upper East', 'Upper West', 'Lower East', 'Herald Square',
        'Penn Station', 'Grand Central', 'JFK', 'LaGuardia', 'Empire State'
    ]

    nyc_pattern = '|'.join(nyc_indicators)
    nyc_filtered = df_for_filtering[df_for_filtering['hotel_name'].str.contains(nyc_pattern, case=False, na=False)]

    # Remove identified London hotels
    london_hotels_to_remove = ['The Soho Hotel', 'The Z Hotel Soho', 'hub by Premier Inn London Soho hotel']
    final_clean_df = nyc_filtered[~nyc_filtered['hotel_name'].isin(london_hotels_to_remove)].copy()

    # Save to silver staging directory
    final_clean_df.to_parquet(output_file, compression="snappy")

    print(f"Final dataset saved")
    print(f"Location: {output_file}")
    file_size_mb = output_file.stat().st_size / (1024*1024)
    print(f"Rows: {len(final_clean_df):,}")
    print(f"Hotels: {final_clean_df['hotel_name'].nunique()}")
    print(f"File size: {file_size_mb:.1f} MB")
    print(f"\nReady for gold layer processing and analysis")

## 8. Final Verification & Cleanup

*Verify saved dataset and optional cleanup of intermediate files*

In [ ]:
# Print comprehensive final summary
print_final_summary(
    output_file,
    dataset_name="TripAdvisor NYC 2022-2025",
    file_type='parquet'
)

# Print storage breakdown
bronze_base = project_root / "data" / "bronze" / "tripadvisor"
print_storage_summary(
    bronze_base,
    silver_staging,
    dataset_name="TripAdvisor NYC"
)

## 9. Next Steps: Gold Layer Processing

**Current Status:**
Bronze → Silver workflow complete for TripAdvisor NYC dataset

**Upcoming Gold Layer Integration:**

- **Multi-dataset analysis:** All processed silver datasets (TripAdvisor NYC, Yelp New Orleans, AirBnB LA/Chicago) will be explored for shared columns
  
- **Schema standardization:** Common fields (location, date, rating, text) will be unified across datasets
  
- **Data quality:** Null value handling, strategic imputation, and appropriate data type conversions
  
- **Analysis-ready format:** Final gold datasets optimized for sentiment analysis and tourism correlation modeling

**Gold Processing Pipeline:**
1. Load all silver datasets and analyze column overlap
2. Standardize shared column names and formats
3. Handle missing values with dataset-appropriate strategies
4. Convert data types for analysis efficiency
5. Create unified gold datasets for cross-platform analysis
